# Astronomical Source Detection: f105w Band Image
## (HST image analysis)
By Nikos Mynhier

## Import packages and set rcParam


In [ ]:
import numpy as np
import sep
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline 

rcParams['figure.figsize'] = [10.,8.]

## Read in image data
## Set mean and standard deviation
## Show image with corrected display parameters

In [ ]:
#Read in image data
image_data = get_pkg_data_filename("/Users/nikosmynhier/Desktop/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data = fits.getdata(image_data, ext=0)
data = data.byteswap().newbyteorder()

m = np.mean(data)   #Create variable for mean data
s = np.std(data)    #Create variable for standard deviation

#Show image with corrected display parameters
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

#Save image
plt.savefig('data.png')

## Use sep package to create a variable for background data

In [ ]:
bkg = sep.Background(data)

## Print the mean/noise of background data

In [ ]:
print(bkg.globalback)
print(bkg.globalrms)

## Create background as a 2D array with same image size

In [ ]:
bkg_image = bkg.back()

## Show image of background points

In [ ]:
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('bkg105_image.png')

## Create a variable for the rms of the background data

In [ ]:
bkg_rms = bkg.rms()

## Show image of the rms of background points

In [ ]:
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar()
plt.savefig('bkg_rms.png')

## Subtract the background data from the orginal data

In [ ]:
data_sub = data - bkg

## Show the data without the background

In [ ]:
plt.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()
plt.savefig('data-bkg.png')

## Identify all objects from data and print number of objects

In [ ]:
#changed the factor of recognition to 5 instead of 1.5 due to excessive objects 
objects = sep.extract(data_sub, 5, err=bkg.globalrms)

l = len(objects)
s = "%g objects were found in this image" % l
print(s)

## Show all objects with circles that represent their ellipiticity

In [ ]:
from matplotlib.patches import Ellipse

fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')

for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]), width=6*objects['a'][i], height=6*objects['b'][i], angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
    
plt.savefig('detected_objects.png')

## Find the flux, flux error, and flag based off identified circles

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'], 3.0, err=bkg.globalrms, gain=1.0)

## Show the first ten values of flux and flux error

In [ ]:
for i in range(10):    #show flux of ten objects out of total
    
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

## Convert Flux to AB units

In [ ]:
flux_v = flux * (3.63 * (10**-20) ) # An object with Fν = 3.63 x 10-20 erg cm-2 s-1 Hz-1 will have magnitude AB =0 in every filter.

fluxAB = -2.5* np.log(flux_v) - 48.60  # ABmag = -2.5 log Fν - 48.6 where its zero point is 48.6

## Plot a histogram of the Flux (in AB) vs Frequency

In [ ]:
histogram = plt.hist(fluxAB, bins='auto', range=(0,100))
plt.ylim([0,320])
plt.xlim([0,90])
plt.xlabel(['Flux (AB)'])
plt.ylabel(['Frequency'])
plt.title(['Histogram of Flux (in AB) vs Frequency'])
plt.show()

## This ends the astronomical source detection for the f105w band image

# Extra Credit: 3-Color False Image
Coauthored by Nikos Mynhier and Eric Ryan

In [ ]:
import numpy as np
import sep
from astropy.utils.data import get_pkg_data_filename
from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline 

rcParams['figure.figsize'] = [10.,10.]

## Import the f105w image

In [ ]:
image_data105 = get_pkg_data_filename("/Users/ericryan/Downloads/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data105 = fits.getdata(image_data105, ext=0)
data105 = data105.byteswap().newbyteorder()

## Identify and remove the background

In [ ]:
bkg105 = sep.Background(data105)
bkg105_image = bkg105.back()
bkg105_rms = bkg105.rms()
data_sub105 = data105 - bkg105

## Import the f125w image

In [ ]:
image_data125 = get_pkg_data_filename("/Users/ericryan/Downloads/hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits")
data125 = fits.getdata(image_data125, ext=0)
data125 = data125.byteswap().newbyteorder()

## Identify and remove the background

In [ ]:
bkg125 = sep.Background(data125)
bkg125_image = bkg125.back()
bkg125_rms = bkg125.rms()
data_sub125 = data125 - bkg125

## Import the f160w image

In [ ]:
image_data160 = get_pkg_data_filename("/Users/ericryan/Downloads/hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits")
data160 = fits.getdata(image_data160, ext=0)
data160 = data160.byteswap().newbyteorder()

## Identify and remove the background

In [ ]:
bkg160 = sep.Background(data160)
bkg160_image = bkg160.back()
bkg160_rms = bkg160.rms()
data_sub160 = data160 - bkg160

## Set values for r, g, and b

In [ ]:
data_sub105_min = data_sub105.min()
data_sub105_max = data_sub105.max()

b = 41.6*(data_sub105 - data_sub105_min)/(data_sub105_max - data_sub105_min)
print(b)   #to check the intensity of blue channel

In [ ]:
data_sub125_min = data_sub125.min()
data_sub125_max = data_sub125.max()

g = 3449*(data_sub125 - data_sub125_min)/(data_sub125_max - data_sub125_min) 
print(g)   # to check the intensity of green channel

In [ ]:
data_sub160_min = data_sub160.min()
data_sub160_max = data_sub160.max()

r = 41.6*(data_sub160 - data_sub160_min)/(data_sub160_max-data_sub160_min) 
print(r)   # to check the intensity of red channel

## Convert to RGB image

In [ ]:
import matplotlib.colors as colors

rgb_image = np.zeros((3600,3600,3))

rgb_image[:,:,0] = r
rgb_image[:,:,1] = g
rgb_image[:,:,2] = b

m, s = np.mean(rgb_image), np.std(rgb_image)

## Show final image

In [ ]:
plt.imshow(rgb_image, interpolation='nearest', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar()

### This ends the extra credit